In [4]:
!conda install cudatoolkit
!pip install cuda-python

/bin/bash: conda: command not found
     |████████████████████████████████| 15.4 MB 10.8 MB/s 


In [5]:
!pip install numba
!find /- iname 'libdevice'
!find /- iname 'libnvvm.so'

find: ‘/-’: No such file or directory
find: ‘iname’: No such file or directory
find: ‘libdevice’: No such file or directory
find: ‘/-’: No such file or directory
find: ‘iname’: No such file or directory
find: ‘libnvvm.so’: No such file or directory


In [8]:
#install libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math
from numba import jit,njit,vectorize,cuda,uint8,f8,uint32
from pylab import imshow, show
from timeit import default_timer as timer

# Cuda Vectorize

In [11]:
from numba import (cuda, vectorize)
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from functools import wraps
from time import time

In [40]:
da=pd.read_csv('aug_train.csv')
da.head(5)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [43]:
#Let's Make A Function
def train():
    df=pd.read_csv('aug_train.csv')
    df=df[['enrollee_id','city_development_index','training_hours','target']]
    df.dropna()
    df['city_development_index']=df['city_development_index'].map(lambda x: "{:.0f}".format(x))
    df['target']=df['target'].map(lambda x: "{:.0f}".format(x))
    return df

def train_array():
    df=train()
    return np.float32(df)

@vectorize(['float32(float32,float32)'],target='cuda')
def add_ufunc(x,y):
    return x+y

def cuda_operation():
    """Performs Vectorized Opearations on GPU"""
    
    x=train_array()
    y=train_array()
    
    print('Movement Calculations From CPU to GPU!!!')
    x_device=cuda.to_device(x)
    y_device=cuda.to_device(y)
    out_device=cuda.device_array(shape=(x_device.shape[0],x_device.shape[1]),dtype=np.float32)
    print(x_device)
    print(x_device.shape)
    print(x_device.dtype)
    
    print('Calculating On GPU')
    add_ufunc(x_device,y_device,out=out_device)
    
    out_host=out_device.copy_to_host()
    print(f"Calculations from GPU {out_host}")
cuda_operation()    

Movement Calculations From CPU to GPU!!!
(19158, 4)
float32
Calculating On GPU
Calculations from GPU [[1.7898e+04 2.0000e+00 7.2000e+01 2.0000e+00]
 [5.9450e+04 2.0000e+00 9.4000e+01 0.0000e+00]
 [2.3122e+04 2.0000e+00 1.6600e+02 0.0000e+00]
 ...
 [4.9152e+04 2.0000e+00 8.8000e+01 0.0000e+00]
 [1.1512e+04 2.0000e+00 1.9400e+02 0.0000e+00]
 [4.7668e+04 2.0000e+00 2.5400e+02 0.0000e+00]]
